In [2]:
pip install scikit-learn==1.0.2

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Manipulación de datos y visualización
import pandas as pd
import matplotlib.pyplot as plt

# División de datos y preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Modelos básicos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Métodos de ensamblaje (Ensemble)
from sklearn.ensemble import VotingClassifier, StackingClassifier, BaggingClassifier, AdaBoostClassifier

# Búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

# Evaluación de modelos
from sklearn.metrics import f1_score

#Libreria LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Librería Lazy Predict (para comparación rápida de modelos)
from lazypredict.Supervised import LazyClassifier




/home/jupyter-user6/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Se importan los datasets

In [2]:
#Ciprofloxacin Loose
ResistanceCiprofloxacinLoose= pd.read_csv("https://github.com/ccm-bioinfo/Camda24_resistance/raw/refs/heads/main/DataSets/ResistanceCiprofloxacinLoose.tsv.gz", sep='\t',dtype={3: str})

#Ciprofloxacin Strict
ResistanceCiprofloxacinStrict=pd.read_csv("https://github.com/ccm-bioinfo/Camda24_resistance/raw/refs/heads/main/DataSets/ResistanceCiprofloxacinStrict.tsv.gz", sep='\t',dtype={3: str})

#Meropenem Loose
ResistanceMeropenemLoose=pd.read_csv("https://github.com/ccm-bioinfo/Camda24_resistance/raw/refs/heads/main/DataSets/ResistanceMeropenemLoose.tsv.gz", sep='\t',dtype={3: str})

#Meropenem Strict
ResistanceMeropenemStrict=pd.read_csv("https://github.com/ccm-bioinfo/Camda24_resistance/raw/refs/heads/main/DataSets/ResistanceMeropenemStrict.tsv.gz", sep='\t',dtype={3: str})

In [96]:
# Filtrar datos que faltan en la columna "phenotype" (datos que necesitamos predecir)
ResistanceCiprofloxacinLoose_to_predict = ResistanceCiprofloxacinLoose[ResistanceCiprofloxacinLoose["phenotype"].isna()]
ResistanceCiprofloxacinStrict_to_predict = ResistanceCiprofloxacinStrict[ResistanceCiprofloxacinStrict["phenotype"].isna()]
ResistanceMeropenemLoose_to_predict = ResistanceMeropenemLoose[ResistanceMeropenemLoose["phenotype"].isna()]
ResistanceMeropenemStrict_to_predict = ResistanceMeropenemStrict[ResistanceMeropenemStrict["phenotype"].isna()]

# Filtrar datos válidos (solo los que tienen "Resistant" o "Susceptible" como valores en "phenotype")
RCL = ResistanceCiprofloxacinLoose[ResistanceCiprofloxacinLoose["phenotype"].isin(["Resistant", "Susceptible"])]
RCS= ResistanceCiprofloxacinStrict[ResistanceCiprofloxacinStrict["phenotype"].isin(["Resistant", "Susceptible"])]
RML= ResistanceMeropenemLoose[ResistanceMeropenemLoose["phenotype"].isin(["Resistant", "Susceptible"])]
RMS= ResistanceMeropenemStrict[ResistanceMeropenemStrict["phenotype"].isin(["Resistant", "Susceptible"])]

# Ensamblaje para Ciprofloxacin y Meropenem con las tablas RCL y RML 

In [6]:
RCL

,accession,genus,species,phenotype,mic,3000535,3005091,3000833,3003665,3007433,...,3003285-S531G,3007051-I572F,3007051-I837V,3003294-D105E,3003394-S66P,3005106-A352E,3003937-N514H,3003937-L546V,3003304-E540V,3003304-E504V
0,SRR3138666,Campylobacter,jejuni,Susceptible,0.12,19.00,10.00,4.00,3.00,3.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,SRR3138667,Campylobacter,jejuni,Susceptible,0.06,19.00,6.00,7.00,4.00,4.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,SRR3138668,Campylobacter,jejuni,Susceptible,0.06,16.00,7.00,5.00,3.00,3.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,SRR3138669,Campylobacter,jejuni,Susceptible,0.06,16.00,7.00,5.00,3.00,3.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,SRR3138670,Campylobacter,jejuni,Susceptible,0.06,20.00,8.00,6.00,4.00,4.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813,SRR4449966,Salmonella,enterica,Susceptible,0.01,25.00,18.00,30.00,22.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2814,SRR4449967,Salmonella,enterica,Susceptible,0.01,24.00,18.00,29.00,21.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2815,SRR4449972,Salmonella,enterica,Susceptible,0.01,24.00,18.00,28.00,22.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2816,SRR4449973,Salmonella,enterica,Susceptible,0.01,26.00,18.00,29.00,23.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
RML

,accession,genus,species,phenotype,mic,3000535,3005091,3000833,3003665,3007433,...,3003285-S531G,3007051-I572F,3007051-I837V,3003294-D105E,3003394-S66P,3005106-A352E,3003937-N514H,3003937-L546V,3003304-E540V,3003304-E504V
0,GCA_002947415,Acinetobacter,baumannii,Resistant,8.00,20.00,7.00,19.00,19.00,16.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,GCA_002947845,Acinetobacter,baumannii,Resistant,8.00,18.00,7.00,18.00,18.00,16.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,GCA_002948925,Acinetobacter,baumannii,Resistant,8.00,19.00,6.00,15.00,18.00,14.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,GCA_002996805,Acinetobacter,baumannii,Resistant,8.00,18.00,7.00,19.00,18.00,15.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,GCA_003006035,Acinetobacter,baumannii,Resistant,8.00,20.00,8.00,17.00,21.00,16.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129,SRR5122327,Pseudomonas,aeruginosa,Resistant,8.00,44.00,20.00,34.00,50.00,30.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3130,SRR5122329,Pseudomonas,aeruginosa,Resistant,8.00,42.00,23.00,34.00,48.00,28.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3131,SRR5122330,Pseudomonas,aeruginosa,Resistant,8.00,47.00,24.00,34.00,50.00,30.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3132,SRR5122331,Pseudomonas,aeruginosa,Resistant,8.00,43.00,25.00,35.00,47.00,29.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
def encode_labels(dataframe):
    """
    Encodes the labels in the specified column of the dataframe.
    
    Parameters:
    - dataframe: The input dataframe.
    
    Returns:
    - encoded_labels: The encoded labels.
    - label_encoder: The label encoder used for encoding.
    """
    labels = dataframe.iloc[:, 3]
    label_encoder = LabelEncoder()
    
    # Fit and transform the labels
    encoded_labels = label_encoder.fit_transform(labels)
    
    return encoded_labels, label_encoder

# Example usage
# Assuming DF is your DataFrame
#encoded_labels, label_encoder = encode_labels(DF)
#print(encoded_labels)

In [9]:
def custom_train_test_split(dataframe, labels, test_size=0.2, random_state=42):
    """
    Splits the dataframe into training and testing sets.
    
    Parameters:
    - dataframe: The input dataframe.
    - labels: The labels to be used for stratification.
    - test_size: The proportion of the dataset to include in the test split.
    - random_state: Controls the shuffling applied to the data before applying the split.
    
    Returns:
    - X_train, X_test, y_train, y_test: The split datasets.
    """
    stratify_labels = labels
    
    X_train, X_test, y_train, y_test = train_test_split(
        dataframe.iloc[:, 5:],  # Splitting columns from 6 onward
        labels,
        stratify=stratify_labels,
        train_size=1 - test_size,
        random_state=random_state,
        shuffle=True
    )
    
    return X_train, X_test, y_train, y_test

# Example usage:
# DF is your dataframe, and encoded_labels are your encoded labels
#X_train, X_test, y_train, y_test = custom_train_test_split(DF, encoded_labels)

Dividimos nuestro set de datos para hacer validación cruzada

In [10]:
#Ciprofloxacin
encoded_labels, label_encoder = encode_labels(RCL)
X_train, X_test, y_train, y_test = custom_train_test_split(RCL, encoded_labels)

In [11]:
#Meropenem
encoded_labels_M, label_encoder_M = encode_labels(RML)
X_train_M, X_test_M, y_train_M, y_test_M = custom_train_test_split(RML, encoded_labels_M)

Estandarizamos los datos

In [12]:
#Ciprofloxacin
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#Meropenem
scaler = StandardScaler()
scaler.fit(X_train_M)
X_train_M = scaler.transform(X_train_M)
X_test_M = scaler.transform(X_test_M)


En la siguiente celda, utilizaremos **LazyClassifier** para entrenar varios modelos de clasificación con el conjunto de datos de RCL.

In [14]:
#Ciprofloxacin
fast_models = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models,predictions = fast_models.fit(X_train, X_test, y_train, y_test)
models

 97%|█████████▋| 28/29 [03:11<00:06,  6.56s/it]

[LightGBM] [Info] Number of positive: 2083, number of negative: 171
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030929 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15212
[LightGBM] [Info] Number of data points in the train set: 2254, number of used features: 2812
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.924135 -> initscore=2.499901
[LightGBM] [Info] Start training from score 2.499901
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


100%|██████████| 29/29 [03:12<00:00,  6.65s/it]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BaggingClassifier,1.00,0.99,0.99,1.00,2.57
DecisionTreeClassifier,0.99,0.98,0.98,0.99,0.83
RidgeClassifierCV,0.99,0.98,0.98,0.99,14.53
LGBMClassifier,0.99,0.97,0.97,0.99,1.35
AdaBoostClassifier,0.99,0.97,0.97,0.99,5.70
XGBClassifier,0.99,0.97,0.97,0.99,16.87
PassiveAggressiveClassifier,0.99,0.97,0.97,0.99,1.13
LinearSVC,0.99,0.97,0.97,0.99,11.48
LogisticRegression,0.99,0.96,0.96,0.99,1.69


In [15]:
#Meropenem
fast_models_M = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models_M,predictions_M = fast_models.fit(X_train_M, X_test_M, y_train_M, y_test_M)
models_M

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


 97%|█████████▋| 28/29 [04:48<00:12, 12.74s/it]

[LightGBM] [Info] Number of positive: 1573, number of negative: 934
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 23095
[LightGBM] [Info] Number of data points in the train set: 2507, number of used features: 3615
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.627443 -> initscore=0.521263
[LightGBM] [Info] Start training from score 0.521263


100%|██████████| 29/29 [04:50<00:00, 10.02s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.90,0.88,0.88,0.90,1.77
XGBClassifier,0.90,0.88,0.88,0.90,30.64
AdaBoostClassifier,0.88,0.86,0.86,0.88,6.99
ExtraTreesClassifier,0.87,0.86,0.86,0.87,2.51
RandomForestClassifier,0.87,0.85,0.85,0.87,2.13
BaggingClassifier,0.85,0.85,0.85,0.86,11.84
DecisionTreeClassifier,0.85,0.85,0.85,0.85,2.42
LogisticRegression,0.85,0.85,0.85,0.85,1.69
PassiveAggressiveClassifier,0.85,0.84,0.84,0.85,2.09


## Optimización de parámetros

### K-nearest neighbors (Grid Search)

In [16]:
parameters = {'n_neighbors':[2*x+1 for x in range(10)], 
              'weights':['uniform', 'distance'], 
              'algorithm':['ball_tree', 'kd_tree', 'brute'], 
              'metric':['minkowski', 'manhattan', 'euclidean']
             }
knn = KNeighborsClassifier()
# Optimización para el primer conjunto de datos pertenecientes  a Ciprofloxacin
grid_search_knn_1 = GridSearchCV(KNeighborsClassifier(), parameters, scoring='f1_micro', cv=5, n_jobs=-1)
grid_search_knn_1.fit(X_train, y_train)
best_knn_1 = grid_search_knn_1.best_estimator_

In [20]:
# Optimización para el segundo conjunto de datos pertenecientes a Meropenem
grid_search_knn_2 = GridSearchCV(KNeighborsClassifier(), parameters, scoring='f1_micro', cv=5, n_jobs=-1)
grid_search_knn_2.fit(X_train_M, y_train_M)
best_knn_2 = grid_search_knn_2.best_estimator_

In [17]:
#Ciprofloxacin
grid_search_knn_1.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', 'manhattan', 'euclidean'],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'weights': ['uniform', 'distance']},
             scoring='f1_micro')

In [21]:
#Meropenem 
grid_search_knn_2.fit(X_train_M, y_train_M)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski', 'manhattan', 'euclidean'],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'weights': ['uniform', 'distance']},
             scoring='f1_micro')

In [22]:
pd.DataFrame(grid_search_knn_1.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.47,0.20,19.36,1.75,ball_tree,minkowski,1,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.98,0.98,0.97,0.97,0.97,0.00,16
1,1.54,0.19,20.30,1.28,ball_tree,minkowski,1,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.98,0.98,0.97,0.97,0.97,0.00,16
2,1.52,0.19,19.11,1.51,ball_tree,minkowski,3,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.97,0.96,0.97,0.98,0.97,0.01,37
3,1.65,0.13,20.37,0.51,ball_tree,minkowski,3,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.97,0.96,0.97,0.98,0.97,0.01,46
4,1.73,0.50,19.28,1.67,ball_tree,minkowski,5,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.96,0.96,0.96,0.98,0.97,0.01,76


In [23]:
pd.DataFrame(grid_search_knn_1.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.47,0.20,19.36,1.75,ball_tree,minkowski,1,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.98,0.98,0.97,0.97,0.97,0.00,16
1,1.54,0.19,20.30,1.28,ball_tree,minkowski,1,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.98,0.98,0.97,0.97,0.97,0.00,16
2,1.52,0.19,19.11,1.51,ball_tree,minkowski,3,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.97,0.96,0.97,0.98,0.97,0.01,37
3,1.65,0.13,20.37,0.51,ball_tree,minkowski,3,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.97,0.96,0.97,0.98,0.97,0.01,46
4,1.73,0.50,19.28,1.67,ball_tree,minkowski,5,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.97,0.96,0.96,0.96,0.98,0.97,0.01,76


In [24]:
#Mejores parametros para Ciprofloxacin
grid_search_knn_1.best_params_

{'algorithm': 'ball_tree',
 'metric': 'manhattan',
 'n_neighbors': 7,
 'weights': 'distance'}

In [25]:
#Mejores parametros para Meropenem
grid_search_knn_2.best_params_

{'algorithm': 'ball_tree',
 'metric': 'manhattan',
 'n_neighbors': 3,
 'weights': 'distance'}

In [31]:
#Ciprofloxacin
knn_opt_C = grid_search_knn_1.best_estimator_

#Meropenem 
knn_opt_M = grid_search_knn_2.best_estimator_

In [33]:
#Predicciones para Ciprofloxacin
preds_C = knn_opt_C.predict(X_test)

# f1-score en el conjunto de prueba
f1_score(y_test, preds_C, average='micro')

0.973404255319149

In [45]:
from sklearn.metrics import accuracy_score

# Calcula el accuracy en el conjunto de prueba
accuracy_test = accuracy_score(y_test, preds_C)

# Calcula el accuracy en el conjunto de entrenamiento
preds_train_C = knn_opt_C.predict(X_train)
accuracy_train = accuracy_score(y_train, preds_train_C)

print("Accuracy en conjunto de prueba:", accuracy_test)
print("Accuracy en conjunto de entrenamiento:", accuracy_train)

Accuracy en conjunto de prueba: 0.9911347517730497
Accuracy en conjunto de entrenamiento: 1.0


In [35]:
# Predicciones en el conjunto de entrenamiento
preds_train_C = knn_opt_C.predict(X_train)

# f1-score en el conjunto de entrenamiento
f1_score(y_train, preds_train_C, average='micro')

1.0

In [30]:
#Predicciones para Meropenem
preds_M = knn_opt_M.predict(X_test_M)
f1_score(y_test_M, preds_M, average='micro')

0.8484848484848486

In [53]:
# Calcula el accuracy en el conjunto de prueba
accuracy_test_M = accuracy_score(y_test_M, preds_M)

# Calcula el accuracy en el conjunto de entrenamiento
preds_train_M = knn_opt_M.predict(X_train_M)
accuracy_train_M = accuracy_score(y_train_M, preds_train_M)

print("Accuracy en conjunto de prueba:", accuracy_test_M)
print("Accuracy en conjunto de entrenamiento:", accuracy_train_M)

Accuracy en conjunto de prueba: 0.7814992025518341
Accuracy en conjunto de entrenamiento: 0.9996011168727563


### Decision Tree (Random Search)

In [36]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [x for x in range(5, 15)],
    'min_samples_split': [2, 3, 4, 5, 6],
    'min_samples_leaf': [1, 2, 3, 4],   
    'ccp_alpha': [0, 0.01, 0.1, 0.2],
}

In [37]:
tree = DecisionTreeClassifier()
#Ciprofloxacin
random_search_tree_C= RandomizedSearchCV(tree, param_grid, n_iter = 100, scoring = 'f1_micro', cv = 5, n_jobs = 2, random_state=42)
#Meropenem
random_search_tree_M= RandomizedSearchCV(tree, param_grid, n_iter = 100, scoring = 'f1_micro', cv = 5, n_jobs = 2, random_state=42)

In [38]:
#Ciprofloxacin
random_search_tree_C.fit(X_train, y_train)
random_search_tree_C.best_params_

{'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_depth': 6,
 'criterion': 'entropy',
 'ccp_alpha': 0.2}

In [39]:
#Meropenem
random_search_tree_M.fit(X_train_M, y_train_M)
random_search_tree_M.best_params_

{'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_depth': 6,
 'criterion': 'entropy',
 'ccp_alpha': 0.2}

In [42]:
tree_opt_C = random_search_tree_C.best_estimator_
tree_opt_M = random_search_tree_M.best_estimator_

In [47]:
#CIPROFLOXACIN
preds_t_C = tree_opt_C.predict(X_test)
f1_score(y_test, preds_t_C, average='micro')# Predicciones en el conjunto de entrenamiento
preds_train_C = knn_opt_C.predict(X_train)

# f1-score en el conjunto de entrenamiento
f1_train = f1_score(y_train, preds_train_C, average='micro')
print(f"f1-score en el conjunto de entrenamiento: {f1_train}")

# f1-score en el conjunto de prueba
f1_test = f1_score(y_test, preds_t_C, average='micro')
print(f"f1-score en el conjunto de prueba: {f1_test}")

f1-score en el conjunto de entrenamiento: 1.0
f1-score en el conjunto de prueba: 0.9911347517730497


In [63]:
# Calcula el accuracy en el conjunto de prueba
accuracy_score(y_test, preds_t_C)

0.9911347517730497

In [46]:
#MEROPENEM
preds_t_M = tree_opt_M.predict(X_test_M)
f1_score(y_test_M, preds_t_M, average='micro')

0.7814992025518342

In [65]:
# Calcula el accuracy en el conjunto de prueba
accuracy_score(y_test_M, preds_t_M)

0.7814992025518341

### Support Vector Machine (Bayesian Optimization)

In [50]:
bayesian_opt_C = BayesSearchCV(
    SVC(),
    {
        'C': (1e-3, 1e+3, 'log-uniform'),
        'gamma': (1e-3, 1e+1, 'log-uniform'),
        'degree': (1, 6),
        'kernel': ['poly', 'rbf'], 
    },
    scoring = 'f1_micro',
    n_iter=21,
    n_points = 3,
    cv=5,
    n_jobs = 5,
    random_state = 42
    
)

In [51]:
bayesian_opt_M = BayesSearchCV(
    SVC(),
    {
        'C': (1e-3, 1e+3, 'log-uniform'),
        'gamma': (1e-3, 1e+1, 'log-uniform'),
        'degree': (1, 6),
        'kernel': ['poly', 'rbf'], 
    },
    scoring = 'f1_micro',
    n_iter=21,
    n_points = 3,
    cv=5,
    n_jobs = 5,
    random_state = 42
    
)

In [52]:
#Ciprofloxacin
bayesian_opt_C.fit(X_train, y_train)
#Meropenem
bayesian_opt_M.fit(X_train_M, y_train_M)

BayesSearchCV(cv=5, estimator=SVC(), n_iter=21, n_jobs=5, n_points=3,
              random_state=42, scoring='f1_micro',
              search_spaces={'C': (0.001, 1000.0, 'log-uniform'),
                             'degree': (1, 6),
                             'gamma': (0.001, 10.0, 'log-uniform'),
                             'kernel': ['poly', 'rbf']})

In [35]:
#Ciprofloxacin
bayesian_opt_C.best_params_

OrderedDict([('C', 153.0267986386334),
             ('degree', 1),
             ('gamma', 0.0035746551640890665),
             ('kernel', 'poly')])

In [54]:
#Meropenem
bayesian_opt_M.best_params_

OrderedDict([('C', 0.001),
             ('degree', 1),
             ('gamma', 0.7100283026277725),
             ('kernel', 'poly')])

In [55]:
svc_opt_C = bayesian_opt_C.best_estimator_
svc_opt_M = bayesian_opt_M.best_estimator_

In [60]:
#Predicciones para Ciprofloxacin 
preds_SVC_C = svc_opt_C.predict(X_test)
f1_score(y_test, preds_SVC_C, average='micro')

0.9911347517730497

In [59]:
#Predicciones para Meropenem
preds_SVC_M = svc_opt_M.predict(X_test_M)
f1_score(y_test_M, preds_SVC_M, average='micro')

0.8660287081339713

## Ensamble de modelos

### Por votación 

In [66]:
eclf1_C= VotingClassifier(estimators=[('knn', knn_opt_C), ('tree', tree_opt_C), ('SVC', svc_opt_C)], voting='hard')
eclf1_M= VotingClassifier(estimators=[('knn', knn_opt_M), ('tree', tree_opt_M), ('SVC', svc_opt_M)], voting='hard')

In [68]:
eclf1_C.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='ball_tree',
                                                   metric='manhattan',
                                                   n_neighbors=7,
                                                   weights='distance')),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.2,
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=4)),
                             ('SVC',
                              SVC(C=153.0267986386334, degree=1,
                                  gamma=0.0035746551640890665,
                                  kernel='poly'))])

In [69]:
eclf1_M.fit(X_train_M, y_train_M)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='ball_tree',
                                                   metric='manhattan',
                                                   n_neighbors=3,
                                                   weights='distance')),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.2,
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=4)),
                             ('SVC',
                              SVC(C=0.001, degree=1, gamma=0.7100283026277725,
                                  kernel='poly'))])

In [41]:
#Ciprofloxacin
preds_E1_C = eclf1_C.predict(X_test)
f1_score(y_test, preds_E1_C, average='micro')

0.9929078014184397

In [71]:
#Meropenem
preds_E1_M =  eclf1_M.predict(X_test_M)
f1_score(y_test_M, preds_E1_M, average='micro')

0.8740031897926634

In [72]:
svc_proba_C = SVC(C=19.958517651170755, gamma = 1.1270954208690063, kernel = 'rbf', probability = True)
svc_proba_M = SVC(C=19.958517651170755, gamma = 1.1270954208690063, kernel = 'rbf', probability = True)

In [73]:
#Ciprofloxacin
svc_proba_C.fit(X_train, y_train)
#Meropenem
svc_proba_M.fit(X_train_M, y_train_M)

SVC(C=19.958517651170755, gamma=1.1270954208690063, probability=True)

In [74]:
eclf2_C = VotingClassifier(estimators=[('knn', knn_opt_C), ('tree', tree_opt_C), ('SVC', svc_proba_C)], voting='soft')
eclf2_M = VotingClassifier(estimators=[('knn', knn_opt_M), ('tree', tree_opt_M), ('SVC', svc_proba_M)], voting='soft')

In [75]:
eclf2_C.fit(X_train, y_train)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='ball_tree',
                                                   metric='manhattan',
                                                   n_neighbors=7,
                                                   weights='distance')),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.2,
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=4)),
                             ('SVC',
                              SVC(C=19.958517651170755,
                                  gamma=1.1270954208690063,
                                  probability=True))],
                 voting='soft')

In [76]:
eclf2_M.fit(X_train_M, y_train_M)

VotingClassifier(estimators=[('knn',
                              KNeighborsClassifier(algorithm='ball_tree',
                                                   metric='manhattan',
                                                   n_neighbors=3,
                                                   weights='distance')),
                             ('tree',
                              DecisionTreeClassifier(ccp_alpha=0.2,
                                                     criterion='entropy',
                                                     max_depth=6,
                                                     min_samples_leaf=4,
                                                     min_samples_split=4)),
                             ('SVC',
                              SVC(C=19.958517651170755,
                                  gamma=1.1270954208690063,
                                  probability=True))],
                 voting='soft')

In [77]:
#CIPROFLOXACIN
preds_E2_C = eclf2_C.predict(X_test)
f1_score(y_test, preds_E2_C, average='micro')

0.9822695035460993

In [83]:
#MEROPENEM
preds_E2_M = eclf2_M.predict(X_test_M)
f1_score(y_test_M, preds_E2_M, average='micro')

0.8484848484848486

### Stacking

In [84]:
eclf3_C = StackingClassifier(
    estimators=[('knn', knn_opt_C), ('tree', tree_opt_C), ('SVC', svc_opt_C)], 
    final_estimator=SVC(kernel='linear')
)
eclf3_M = StackingClassifier(
    estimators=[('knn', knn_opt_M), ('tree', tree_opt_M), ('SVC', svc_opt_M)], 
    final_estimator=SVC(kernel='linear')
)

In [85]:
eclf3_C.fit(X_train, y_train)

StackingClassifier(estimators=[('knn',
                                KNeighborsClassifier(algorithm='ball_tree',
                                                     metric='manhattan',
                                                     n_neighbors=7,
                                                     weights='distance')),
                               ('tree',
                                DecisionTreeClassifier(ccp_alpha=0.2,
                                                       criterion='entropy',
                                                       max_depth=6,
                                                       min_samples_leaf=4,
                                                       min_samples_split=4)),
                               ('SVC',
                                SVC(C=153.0267986386334, degree=1,
                                    gamma=0.0035746551640890665,
                                    kernel='poly'))],
                   final_estimator=SV

In [86]:
eclf3_M.fit(X_train_M, y_train_M)

StackingClassifier(estimators=[('knn',
                                KNeighborsClassifier(algorithm='ball_tree',
                                                     metric='manhattan',
                                                     n_neighbors=3,
                                                     weights='distance')),
                               ('tree',
                                DecisionTreeClassifier(ccp_alpha=0.2,
                                                       criterion='entropy',
                                                       max_depth=6,
                                                       min_samples_leaf=4,
                                                       min_samples_split=4)),
                               ('SVC',
                                SVC(C=0.001, degree=1, gamma=0.7100283026277725,
                                    kernel='poly'))],
                   final_estimator=SVC(kernel='linear'))

In [87]:
#CIPROFLOXACIN
preds_E3_C = eclf3_C.predict(X_test)
f1_score(y_test, preds_E3_C, average='micro')

0.9929078014184397

In [90]:
# Calcula el accuracy en el conjunto de prueba
accuracy_test_C = accuracy_score(y_test, preds_E3_C)

print("Accuracy en conjunto de prueba:", accuracy_test_C)

Accuracy en conjunto de prueba: 0.9929078014184397


In [88]:
#MEROPENEM
preds_E3_M = eclf3_M.predict(X_test_M)
f1_score(y_test_M, preds_E3_M, average='micro')

0.8692185007974481

In [91]:
# Calcula el accuracy en el conjunto de prueba
accuracy_test_M = accuracy_score(y_test_M, preds_E3_M)

print("Accuracy en conjunto de prueba:", accuracy_test_M)

Accuracy en conjunto de prueba: 0.8692185007974481


### Bagging

In [92]:
eclf4_C = BaggingClassifier(base_estimator=svc_opt_C, n_estimators=30, max_samples = 0.8, random_state=0).fit(X_train, y_train)

In [93]:
eclf4_M = BaggingClassifier(base_estimator=svc_opt_M, n_estimators=30, max_samples = 0.8, random_state=0).fit(X_train_M, y_train_M)

In [94]:
#CIPROFLOXACIN
preds_E4_C = eclf4_C.predict(X_test)
f1_score(y_test, preds_E4_C, average='micro')

0.9929078014184397

In [95]:
#MEROPENEM
preds_E4_M = eclf4_M.predict(X_test_M)
f1_score(y_test_M, preds_E4_M, average='micro')

0.8580542264752791